# Форматы данных (1)

Материалы:
* Макрушин С.В. "Лекция 4: Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/library/pickle.html
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

## Лабораторная работа №4

### JSON

In [30]:
import json
import pandas as pd

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [31]:
with open('contributors_sample.json') as f:
    content = f.read()
    
contributors = json.loads(content)
print(contributors[:3])

[{'username': 'uhebert', 'name': 'Lindsey Nguyen', 'sex': 'F', 'address': '01261 Cameron Spring\nTaylorfurt, AK 97791', 'mail': 'jsalazar@gmail.com', 'jobs': ['Energy engineer', 'Engineer, site', 'Environmental health practitioner', 'Biomedical scientist', 'Jewellery designer'], 'id': 35193}, {'username': 'vickitaylor', 'name': 'Cheryl Lewis', 'sex': 'F', 'address': '66992 Welch Brooks\nMarshallshire, ID 56004', 'mail': 'bhudson@gmail.com', 'jobs': ['Music therapist', 'Volunteer coordinator', 'Designer, interior/spatial'], 'id': 91970}, {'username': 'sheilaadams', 'name': 'Julia Allen', 'sex': 'F', 'address': 'Unit 1632 Box 2971\nDPO AE 23297', 'mail': 'darren44@yahoo.com', 'jobs': ['Management consultant', 'Engineer, structural', 'Lecturer, higher education', 'Theatre manager', 'Designer, textile'], 'id': 1848091}]


1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [32]:
print(set([c['address'].split()[0] for c in contributors]))

{'7870', '329', '6483', '2439', '21384', '02432', '59744', '378', '20056', '6285', '11449', '1540', '10771', '768', '8906', '80940', '2089', '41473', '0524', '19194', '94375', '88099', '47121', '96658', '55393', '05236', '95433', '91171', '7239', '3661', '4225', '90261', '4049', '84620', '44772', '481', '7993', '842', '17583', '00043', '58678', '4276', '24395', '045', '20972', '69644', '69648', '590', '7035', '151', '62342', '31089', '49118', '96495', '05486', '9324', '60752', '486', '927', '66348', '94664', '089', '42614', '50316', '1500', '16178', '733', '07663', '781', '94319', '4474', '05695', '72752', '42423', '564', '66696', '7325', '31836', '77075', '54197', '554', '75553', '4977', '3590', '16311', '25739', '8716', '3585', '2842', '64686', '09756', '14481', '18692', '533', '044', '03089', '6197', '500', '3818', '2164', '85231', '18988', '4994', '63519', '661', '76489', '6305', '3829', '1255', '3178', '693', '84772', '20984', '023', '45875', '05048', '7447', '37922', '28515', '00

1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [33]:
def find_by_username(username: str):
    for contributor in contributors:
        if contributor['username'] == username:
            return contributor
    raise ValueError("Contributor doesn't exist")

print(find_by_username('uhebert'))    # ищем существующего пользователя
print(find_by_username('kfjkajl'))    # ищем несуществующего пользователя

{'username': 'uhebert', 'name': 'Lindsey Nguyen', 'sex': 'F', 'address': '01261 Cameron Spring\nTaylorfurt, AK 97791', 'mail': 'jsalazar@gmail.com', 'jobs': ['Energy engineer', 'Engineer, site', 'Environmental health practitioner', 'Biomedical scientist', 'Jewellery designer'], 'id': 35193}


ValueError: Contributor doesn't exist

1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [34]:
print(f'Количество женщин: {len([c for c in contributors if c["sex"] == "F"])}')
print(f'Количество мужчин: {len([c for c in contributors if c["sex"] == "M"])}')

Количество женщин: 2136
Количество мужчин: 2064


1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [35]:
pd_contributors = pd.read_json('contributors_sample.json').drop(['name', 'address', 'mail', 'jobs'], axis=1)
pd_contributors

,username,sex,id
0,uhebert,F,35193
1,vickitaylor,F,91970
2,sheilaadams,F,1848091
3,nicole82,F,50969
4,jean67,M,676820
...,...,...,...
4195,stevenspencer,F,423555
4196,rwilliams,M,35251
4197,lmartinez,F,135887
4198,brendahill,M,212714


1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [36]:
recipes = pd.read_csv('recipes_sample.csv')
contributors_and_recipes = pd.merge(
                                recipes,
                                pd_contributors,
                                left_on='contributor_id',
                                right_on='id',
                                how='left'
                            ).drop(['id_x', 'id_y'], axis=1)

contributors_and_recipes[contributors_and_recipes['username'].isna()].shape[0]

15059

### pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [37]:
job_people = {}
for contributor in contributors:
    for job in contributor['jobs']:
        job_people.setdefault(job, []).append(contributor['username'])
print(job_people)

{'Energy engineer': ['uhebert', 'annmoore', 'garysilva', 'martinezashley', 'sextonsheila', 'pjames', 'smithjonathan', 'wardjames', 'cwheeler', 'ucarlson', 'robert71', 'johnsontheresa', 'amanda41', 'stacey47', 'timothynelson', 'timothynelson', 'rogersmichael', 'melissa94', 'wmcdaniel', 'charles74', 'smithjennifer', 'clintonjones'], 'Engineer, site': ['uhebert', 'nancy12', 'andrea03', 'catherineross', 'wesley32', 'natalieross', 'rossdoris', 'christophersmith', 'dbooker', 'ericarobertson', 'trantricia', 'tpugh', 'jasonvelez', 'samantha36', 'brandidaniels', 'tenglish', 'reyesbrett', 'austin18', 'vjohnson', 'zmejia', 'daniel04', 'cynthia20', 'morgan15', 'avaldez', 'jessica92', 'laurieholloway', 'baileyvictoria'], 'Environmental health practitioner': ['uhebert', 'jonathanchristian', 'xjohnson', 'dsmith', 'james01', 'nancytaylor', 'ztaylor', 'andrewwoods', 'susan54', 'fmaldonado', 'james74', 'bakerjacob', 'stephanie81', 'whitejoseph', 'qolson', 'hknox', 'gonzalesdaniel', 'tranronald', 'jesseg

2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

In [38]:
import pickle
import os

with open('job_people.pickle', 'wb') as p_file:
    pickle.dump(job_people, p_file)
    
with open('job_people.json', 'w') as j_file:
    json.dump(job_people, j_file, indent=4)
    
print(os.stat('job_people.pickle').st_size)
print(os.stat('job_people.json').st_size)

132272
389655


2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

In [39]:
with open('job_people.pickle', 'rb') as pc_file:
    print(pickle.load(pc_file))

{'Energy engineer': ['uhebert', 'annmoore', 'garysilva', 'martinezashley', 'sextonsheila', 'pjames', 'smithjonathan', 'wardjames', 'cwheeler', 'ucarlson', 'robert71', 'johnsontheresa', 'amanda41', 'stacey47', 'timothynelson', 'timothynelson', 'rogersmichael', 'melissa94', 'wmcdaniel', 'charles74', 'smithjennifer', 'clintonjones'], 'Engineer, site': ['uhebert', 'nancy12', 'andrea03', 'catherineross', 'wesley32', 'natalieross', 'rossdoris', 'christophersmith', 'dbooker', 'ericarobertson', 'trantricia', 'tpugh', 'jasonvelez', 'samantha36', 'brandidaniels', 'tenglish', 'reyesbrett', 'austin18', 'vjohnson', 'zmejia', 'daniel04', 'cynthia20', 'morgan15', 'avaldez', 'jessica92', 'laurieholloway', 'baileyvictoria'], 'Environmental health practitioner': ['uhebert', 'jonathanchristian', 'xjohnson', 'dsmith', 'james01', 'nancytaylor', 'ztaylor', 'andrewwoods', 'susan54', 'fmaldonado', 'james74', 'bakerjacob', 'stephanie81', 'whitejoseph', 'qolson', 'hknox', 'gonzalesdaniel', 'tranronald', 'jesseg

### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

In [40]:
from bs4 import BeautifulSoup

with open('steps_sample.xml') as f:
    data = BeautifulSoup(f, 'xml')
    
recipes_and_steps = {}
for recipe in data.recipes.find_all('recipe'):
    steps = [step.next for step in recipe.steps.find_all('step')]
    recipes_and_steps[f'{recipe.find("id").next}'] = steps

with open('steps_sample.json', 'w') as file:
    json.dump(recipes_and_steps, file, indent=4)

3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

In [41]:
steps_amount = {}
for recipe_id, steps in recipes_and_steps.items():
    steps_amount.setdefault(len(steps), []).append(recipe_id)
    
steps_amount

{11: ['44123',
  '302399',
  '375376',
  '140610',
  '374703',
  '111198',
  '257111',
  '432661',
  '114204',
  '63069',
  '165096',
  '33947',
  '250024',
  '330512',
  '315233',
  '25259',
  '331174',
  '407621',
  '263019',
  '112853',
  '383729',
  '13709',
  '336166',
  '143286',
  '387284',
  '290003',
  '370746',
  '34833',
  '11975',
  '426211',
  '373582',
  '88845',
  '456968',
  '14149',
  '507927',
  '73602',
  '91981',
  '175109',
  '390933',
  '193208',
  '83893',
  '243008',
  '259789',
  '303926',
  '410920',
  '446605',
  '32571',
  '74419',
  '308056',
  '78497',
  '111963',
  '361181',
  '302640',
  '356655',
  '53743',
  '57771',
  '420689',
  '74520',
  '50851',
  '176277',
  '266814',
  '27897',
  '189207',
  '138771',
  '279797',
  '177831',
  '32515',
  '256842',
  '95295',
  '383349',
  '109791',
  '332641',
  '116993',
  '173126',
  '187872',
  '177681',
  '249006',
  '314834',
  '283033',
  '117084',
  '49202',
  '284916',
  '247657',
  '313162',
  '424727',

3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

In [42]:
recipes_with_time = []
for recipe in data.recipes.find_all('recipe'):
    minutes = [step.get('has_minutes') for step in recipe.steps.find_all('step')]
    hours = [step.get('has_hours') for step in recipe.steps.find_all('step')]
    if any(minutes) or any(hours):
        recipes_with_time.append(recipe.find('id').next)

print(list(set(recipes_with_time)))    # удалим дубликаты

['426003', '163046', '70204', '61218', '15148', '491517', '390476', '38443', '182182', '422794', '94532', '363299', '231378', '216035', '222328', '96002', '139207', '373728', '141419', '425949', '17969', '105073', '227342', '148365', '199929', '228720', '385030', '264752', '388607', '243871', '129173', '84219', '106897', '132160', '300931', '254694', '4740', '54654', '77640', '182428', '138017', '370850', '148070', '451134', '191859', '10528', '71265', '463469', '218380', '517313', '81593', '19743', '187723', '458309', '447345', '359442', '343461', '469116', '211869', '170543', '378012', '389231', '242373', '302548', '279797', '194647', '382974', '9823', '80617', '111519', '298208', '302640', '290187', '294949', '302151', '19879', '273319', '49860', '332119', '288672', '239972', '23832', '25456', '100622', '50069', '216099', '85583', '37642', '288734', '129581', '433985', '73081', '112057', '132488', '118383', '279215', '68058', '23369', '20192', '290273', '499586', '93186', '129286', 

3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

In [43]:
import pandas as pd

recipes = pd.read_csv('recipes_sample.csv')

n_steps_recipes = {}
for recipe in data.recipes.find_all('recipe'):
    steps = [step.next for step in recipe.steps.find_all('step')]
    n_steps_recipes[f'{recipe.find("id").next}'] = len(steps)
    
recipes_steps = pd.DataFrame(
    {
        'id': n_steps_recipes.keys(),
        'n_steps': n_steps_recipes.values()
    }
).astype('int32')

recipes['n_steps'] = recipes['n_steps'].fillna(recipes_steps['n_steps'])

3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`

In [44]:
print(recipes.n_steps.isna().any())
recipes.n_steps.astype('int')
recipes.to_csv('recipes_sample_with_filled_nsteps.csv')

False
